<a href="https://colab.research.google.com/github/AnnaZhuravleva/avtobreja/blob/master/udpipe_parsing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ufal.udpipe

     |████████████████████████████████| 307kB 2.8MB/s 
  Created wheel for ufal.udpipe: filename=ufal.udpipe-1.2.0.3-cp36-cp36m-linux_x86_64.whl size=5625749 sha256=51c86dd97e3661e66a2f930e58b2f7fde93f58c3949266be52889602f24e4ba7
  Stored in directory: /root/.cache/pip/wheels/0c/9d/db/6d3404c33da5b7adb6c6972853efb6a27649d3ba15f7e9bebb
Successfully built ufal.udpipe


In [2]:
!wget https://github.com/AnnaZhuravleva/avtobreja/raw/master/SentiRuEval_rest_train/SentiRuEval_rest_train.csv


--2019-12-14 21:22:33--  https://github.com/AnnaZhuravleva/avtobreja/raw/master/SentiRuEval_rest_train/SentiRuEval_rest_train.csv
Resolving github.com (github.com)... 192.30.253.113
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/AnnaZhuravleva/avtobreja/master/SentiRuEval_rest_train/SentiRuEval_rest_train.csv [following]
--2019-12-14 21:22:33--  https://raw.githubusercontent.com/AnnaZhuravleva/avtobreja/master/SentiRuEval_rest_train/SentiRuEval_rest_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36571759 (35M) [text/plain]
Saving to: ‘SentiRuEval_rest_train.csv’

SentiRuEval_rest_tr 100%[===================>]  34.88M   133MB/s    in 0.

In [3]:
!wget -O "russian-syntagrus-ud-2.4-190531.udpipe" https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-2998/russian-syntagrus-ud-2.4-190531.udpipe?sequence=74&isAllowed=y

--2019-12-14 21:22:37--  https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-2998/russian-syntagrus-ud-2.4-190531.udpipe?sequence=74
Resolving lindat.mff.cuni.cz (lindat.mff.cuni.cz)... 195.113.20.140
Connecting to lindat.mff.cuni.cz (lindat.mff.cuni.cz)|195.113.20.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45859472 (44M) [application/octet-stream]
Saving to: ‘russian-syntagrus-ud-2.4-190531.udpipe’

russian-syntagrus-u 100%[===================>]  43.73M  17.9MB/s    in 2.4s    

2019-12-14 21:22:41 (17.9 MB/s) - ‘russian-syntagrus-ud-2.4-190531.udpipe’ saved [45859472/45859472]



In [0]:
import sys
import ufal.udpipe

In [0]:
ud_model = ufal.udpipe.Model.load("russian-syntagrus-ud-2.4-190531.udpipe")

In [11]:
help(ufal.udpipe.Pipeline())

Help on built-in function __new__:

__new__(*args, **kwargs) method of builtins.SwigPyObjectType instance
    Create and return a new object.  See help(type) for accurate signature.



In [0]:
pipe = ufal.udpipe.Pipeline(ud_model, "tokenize", "tag", "parse", "conllu")

In [59]:
# Usage example

a = "Положа руку на сердце, очень хочется спать. А не проект по автобрее пилить!"
print(pipe.process(a))

# newdoc
# newpar
# sent_id = 1
# text = Положа руку на сердце, очень хочется спать.
1	Положа	Положать	VERB	_	Aspect=Perf|Tense=Past|VerbForm=Conv|Voice=Act	7	advcl	_	_
2	руку	рука	NOUN	_	Animacy=Inan|Case=Acc|Gender=Fem|Number=Sing	1	obj	_	_
3	на	на	ADP	_	_	4	case	_	_
4	сердце	сердце	NOUN	_	Animacy=Inan|Case=Acc|Gender=Neut|Number=Sing	1	obl	_	SpaceAfter=No
5	,	,	PUNCT	_	_	1	punct	_	_
6	очень	очень	ADV	_	Degree=Pos	7	advmod	_	_
7	хочется	хотеться	VERB	_	Aspect=Imp|Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|Voice=Mid	0	root	_	_
8	спать	спать	VERB	_	Aspect=Imp|VerbForm=Inf|Voice=Act	7	xcomp	_	SpaceAfter=No
9	.	.	PUNCT	_	_	7	punct	_	_

# sent_id = 2
# text = А не проект по автобрее пилить!
1	А	а	CCONJ	_	_	3	cc	_	_
2	не	не	PART	_	_	3	advmod	_	_
3	проект	проект	NOUN	_	Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing	0	root	_	_
4	по	по	ADP	_	_	5	case	_	_
5	автобрее	автобрея	NOUN	_	Animacy=Inan|Case=Dat|Gender=Fem|Number=Sing	6	obl	_	_
6	пилить	пилить	VERB	_	Aspect=Perf|VerbForm=Inf|

In [0]:
import pandas as pd

In [0]:
with open("SentiRuEval_rest_train.csv", 'r') as f:
    train = pd.read_csv(f)

#А то у нас уже записано id
train.drop(["Unnamed: 0"], axis=1, inplace=True)


In [58]:
train.head()

,food,service,text
0,8,8,И пускай на меня не обижается наш прославленны...
1,9,10,"- Здравствуйте. Виа Д’Арженто! - Добрый вечер,..."
2,9,1,"Советую вам уволить Вашего метродотеля Елену, ..."
3,8,9,отличный средне вековый интеръер. Приятное обс...
4,10,8,Ужинали в ресторане Баден-Баден 6 марта . Импо...


In [68]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from os import path, mkdir

d = "/content/drive/My Drive/parsed_train"
mkdir(d)

for i, text in enumerate(train.text):
    parsed_text = open(path.join(d, "{}.conllu".format(i)), 'w', encoding='utf-8')
    parsed = pipe.process(text)
    parsed_text.write(parsed)


Распарсенные тексты [здесь](https://drive.google.com/open?id=181szxLRYHorRrzGTcA1XwwA23MVy7F90)